In [15]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import nltk
import math

In [16]:
def conv_to_char(arr):
    lis = list(arr)
    k = ""
    for i in lis:
        k += str(chr(i))
    return k

In [17]:
location_train_192 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/train"

location_train_224 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-224x224/train"

location_train_331 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-331x331/train"

location_train_512 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-512x512/train"

In [18]:
location_val_192 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/val"

location_val_224 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-224x224/val"

location_val_331 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-331x331/val"

location_val_512 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-512x512/val"

In [19]:
location_test_192 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/test"

location_test_224 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-224x224/test"

location_test_331 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-331x331/test"

location_test_512 = "/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-512x512/test"

In [20]:
def get_file_names(location):   
    loc_files = os.listdir(location)
    file_list = []
    for i in loc_files:
        file_list.append(location+"/"+i)
    return file_list

In [21]:
def get_images(file):
    
    image_description = {
        "id" : tf.io.FixedLenFeature([],tf.string),
        "class" : tf.io.FixedLenFeature([],tf.int64),
        "image" : tf.io.FixedLenFeature([],tf.string)
    }
    y = lambda x : tf.io.parse_single_example(x,image_description)

    ctr = 1    
    
    id = []
    label = []
    image = []

    img_tfrec = tf.data.TFRecordDataset(file)
    img_file = img_tfrec.map(y)
        
    begin = file.rindex("-") + 1
    end = file.rindex(".tfrec")
    no_of_images = int(file[begin:end])
    
    for ef in img_file.take(no_of_images):
        id.append(conv_to_char(ef["id"].numpy()))
        label.append(ef["class"].numpy())
        image.append(tf.io.decode_image(ef["image"],dtype="float64").numpy().tolist())
        print(ctr)
        ctr += 1
    
    return id,label,image

In [22]:
files_train_192 = get_file_names(location_train_192)
files_train_192

['/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/train/07-192x192-798.tfrec',
 '/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/train/02-192x192-798.tfrec',
 '/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/train/06-192x192-798.tfrec',
 '/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/train/03-192x192-798.tfrec',
 '/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/train/05-192x192-798.tfrec',
 '/Users/amith/Documents/GitHub/kaggle/Petals to the Metal - Flower Classification on TPU/tpu-getting-started/tfrecords-jpeg-192x192/train/00-192x192-798.tfrec',
 '/Users/amith/Documents/Git

In [ ]:
id,label,image = get_images(files_train_192[0])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [14]:
id

[<tf.Tensor: shape=(), dtype=string, numpy=b'53b57c02f'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'10d68e426'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'566f0f7b8'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'25993e867'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'15282c04f'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'5ab92ede7'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'974c7754c'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'54c09f9dc'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'69b1bb0d7'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'9f06a6046'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'127b501e6'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'c5da2ef33'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'9718b0c11'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'4b9ad2313'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'6b59ae5f3'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'8ae51fb1d'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'c7021e792'>,
 <tf.Tensor: s